### Part 2: Programming Problems

#### Problem 1 – Line Search
##### (1) Define the function $f(x)$

We consider the function
$$
f(x)= x_1^2+2x_2^2,
$$
where $x=(x_1,x_2)\in\mathbb{R}^2$.

In [7]:
import numpy as np

def f_line(x):
    # x is a numpy array: [x1, x2]
    x1, x2 = x[0], x[1]
    return x1**2 + 2*x2**2

##### (2) Define the gradient of $f(x)$

The gradient is
$$
\nabla f(x)= \begin{pmatrix} 2x_1 \\ 4x_2 \end{pmatrix}.
$$

In [8]:
def grad_f_line(x):
    x1, x2 = x[0], x[1]
    return np.array([2*x1, 4*x2])

##### (3) Evaluate $f(x)$ and $\nabla f(x)$ at $x^{(0)}=(9,1)^T$

In [9]:
x0_line = np.array([9, 1])
print("f(x0) =", f_line(x0_line))
print("grad f(x0) =", grad_f_line(x0_line))

f(x0) = 83
grad f(x0) = [18  4]


##### (4) Define the function $wolfe\_conditions$

This function verifies if the Wolfe conditions hold for a given step size $\alpha>0$. It takes:
- A function $f$
- Its gradient function $\nabla f$
- A current point $x$
- A descent direction $d$
- A step size $\alpha$
- Parameters $\eta$ and $\bar{\eta}$

and returns a tuple of booleans $(\text{first\_wolfe}, \text{second\_wolfe})$.

The Wolfe conditions are:
1. First Wolfe condition (sufficient decrease):
$$
f(x+\alpha d)\le f(x)+\alpha\eta\,\nabla f(x)^T d.
$$
2. Second Wolfe condition (curvature):
$$
\nabla f(x+\alpha d)^T d \ge \bar{\eta}\,\nabla f(x)^T d.
$$

In [10]:
def wolfe_conditions(f, grad_f, x, d, alpha, eta, eta_bar):
    cond1 = f(x + alpha * d) <= f(x) + alpha * eta * np.dot(grad_f(x), d)
    cond2 = np.dot(grad_f(x + alpha * d), d) >= eta_bar * np.dot(grad_f(x), d)
    return (cond1, cond2)
